## 쿨루프 시공 대상 여부 분류
- Tasks
    1. Data Preprocessing
        - 모델 사용을 위해 데이터를 일관성 있게 정리해야 합니다.
    2. Object Detection
        - 전처리 된 데이터를 이용하여 학습을 진행하세요.
        - 인공위성 지도를 이용하여 이미지를 **추가적으로** 10장 수집하고 추론 과정에서 사용하세요.

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.4/750.4 kB 10.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [ ]:
import gdown, zipfile
import os, glob, shutil
from tqdm import tqdm

import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import *

from ultralytics import YOLO, settings
import yaml
from keras.preprocessing import image

### 데이터셋 다운로드 및 압축 해제
- cool_roof_image.zip : 이미지 데이터 압축 파일
- cool_roof_yolo_labels.zip : YOLO 모델 형식에 맞게 가공된 레이블 파일

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def dataset_extract(file_name) :
    with zipfile.ZipFile(file_name, 'r') as zip_ref :
        file_list = zip_ref.namelist()

        if os.path.exists(f'./{file_name[:-4]}/') :
            print(f'데이터셋 폴더가 이미 존재합니다.')
            return

        else :
            for f in tqdm(file_list, desc='Extracting', unit='files') :
                zip_ref.extract(member=f, path=f'./{file_name[:-4]}/')

In [ ]:
dataset_extract('drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/cool_roof_images.zip')

Extracting: 100%|██████████| 200/200 [00:04<00:00, 40.36files/s]


In [ ]:
dataset_extract('drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/cool_roof_yolo_labels.zip')

Extracting: 100%|██████████| 204/204 [00:01<00:00, 121.78files/s]


In [ ]:
dataset_extract('drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/cool_roof_박소영.zip')

Extracting: 100%|██████████| 1303/1303 [00:11<00:00, 118.28files/s]


In [ ]:
dataset_extract('drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/cool_roof_통합(신예린).zip')

Extracting: 100%|██████████| 2048/2048 [00:21<00:00, 96.73files/s] 


### 폴더 생성
- YOLO 모델에서 요구하는 폴더의 형식이 있습니다.
- 해당 형식에 맞춰 폴더를 만드세요.

In [76]:
def make_yolo_dirs(data_path):
    if os.path.exists(data_path):
        shutil.rmtree(data_path)

    if not os.path.exists(data_path + 'train'):
        os.makedirs(data_path + 'train/images')
        os.makedirs(data_path + 'train/labels')

    if not os.path.exists(data_path + 'valid'):
        os.makedirs(data_path + 'valid/images')
        os.makedirs(data_path + 'valid/labels')

In [77]:
data_path = '/content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/data/'
make_yolo_dirs(data_path)

### 데이터 스플릿 & 파일 이동
1. 재현을 위한 난수 고정 : 2024
2. 테스트셋 데이터 20%
- 위 설정에 맞게 데이터를 나누고, 위 과정에서 생성한 폴더에 이동시키세요.

In [78]:
image_paths = sorted(glob.glob("/content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/cool_roof_images/*.jpg"))
txt_paths = sorted(glob.glob("/content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/cool_roof_yolo_labels/obj_train_data/*.txt"))

train_images_paths, valid_images_paths, train_txt_paths, valid_txt_paths = train_test_split(image_paths, txt_paths, test_size=0.2, random_state=2024)

In [79]:
train_images_paths[-3:], train_txt_paths[-3:]

(['/content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/cool_roof_images/roof-75-_jpg.rf.3cd06946a6508f8f95739e1a127026c1.jpg',
  '/content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/cool_roof_images/roof-40-_jpg.rf.34afbefae9406418383e833d8d1fb4d6.jpg',
  '/content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/cool_roof_images/roof-9-_jpg.rf.5f79d80f23e1a934fe2e2cfcd47148d7.jpg'],
 ['/content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/cool_roof_yolo_labels/obj_train_data/roof-75-_jpg.rf.3cd06946a6508f8f95739e1a127026c1.txt',
  '/content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/cool_roof_yolo_labels/obj_train_data/roof-40-_jpg.rf.34afbefae9406418383e833d8d1fb4d6.txt',
  '/content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_p

In [80]:
def file_split(train_images_paths, valid_images_paths, train_txt_paths, valid_txt_paths):
    # Train 이미지와 텍스트 파일을 이동
    for img_path, txt_path in zip(train_images_paths, train_txt_paths):
        shutil.copy(img_path, data_path + 'train/images')
        shutil.copy(txt_path, data_path + 'train/labels')

    # Valid 이미지와 텍스트 파일을 이동
    for img_path, txt_path in zip(valid_images_paths, valid_txt_paths):
        shutil.copy(img_path, data_path + 'valid/images')
        shutil.copy(txt_path, data_path + 'valid/labels')

In [81]:
file_split(train_images_paths, valid_images_paths, train_txt_paths, valid_txt_paths)

In [82]:
len(glob.glob(data_path + 'train/images/*.jpg')), len(glob.glob(data_path + 'train/labels/*txt')), len(glob.glob(data_path + 'valid/images/*.jpg')), len(glob.glob(data_path + 'valid/labels/*txt'))

(760, 760, 191, 191)

In [83]:
glob.glob(data_path + 'train/images/*.jpg')[:3]

['/content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/data/train/images/roof51_png.rf.93c8f1a8cd20c6ac435fe372ecd569d6.jpg',
 '/content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/data/train/images/roof-133-_jpg.rf.e0bd05685d50defc44c659f0c9d872c8.jpg',
 '/content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/data/train/images/roof-44-_jpg.rf.5093328da7d41f46f7638e0e063510e5.jpg']

In [84]:
glob.glob(data_path + 'train/labels/*txt')[:3]

['/content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/data/train/labels/roof51_png.rf.93c8f1a8cd20c6ac435fe372ecd569d6.txt',
 '/content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/data/train/labels/roof-133-_jpg.rf.e0bd05685d50defc44c659f0c9d872c8.txt',
 '/content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/data/train/labels/roof-44-_jpg.rf.5093328da7d41f46f7638e0e063510e5.txt']

### YOLO 모델에 적용할 YAML 생성하기
- 지붕에는 두 가지 형태가 있습니다. 클래스 구분에 주의하세요.
- cool roof
- generic roof

In [85]:
data = {
    "names" : {0 : 'cool roof', 1 : 'generic roof'},
    "nc": 2,
    "path": data_path,
    "train": "train",
    "val": "valid",
    "test": "test",
    }

with open(data_path + 'data.yaml', 'w') as f :
    yaml.dump(data, f)

# # check written file
# with open(data_path + 'data.yaml', 'r') as f :
#     lines = yaml.safe_load(f)
#     print(lines)

### YOLO v8 모델
- yaml 파일의 경로 설정에 주의하세요.

#### 1) 학습 데이터 추가하기 전

In [ ]:
model = YOLO(model='yolov8s.pt', task='detect')

100%|██████████| 21.5M/21.5M [00:00<00:00, 155MB/s]


In [ ]:
model.train(data=data_path + 'data.yaml',
            epochs=200,
            patience=20,
            pretrained=True,
            verbose=True,
            val=True,
            seed=2024,
            )

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/data/data.yaml, epochs=200, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=2024, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=

100%|██████████| 755k/755k [00:00<00:00, 16.6MB/s]

Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

 18                  -1  1    493056  ultralytics.nn.modules.block.C2f             [384, 256, 1]                 
 19                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
 20             [-1, 9]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 21                  -1  1   1969152  ultralytics.nn.modules.block.C2f             [768, 512, 1]                 
 22        [15, 18, 21]  1   2116822  ultralytics.nn.modules.head.Detect           [2, [128, 256, 512]]          
Model summary: 225 layers, 11136374 parameters, 11136358 gradients, 28.6 GFLOPs

Transferred 349/355 items from pretrained weights
TensorBoard: Start with 'tensorboard --logdir runs/detect/train', view at http://localhost:6006/
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...


100%|██████████| 6.23M/6.23M [00:00<00:00, 84.4MB/s]


...

                   all         48        365      0.726      0.676      0.696      0.444
EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 51, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



71 epochs completed in 0.206 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 22.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.33it/s]


                   all         48        365       0.74      0.775      0.799      0.523
             cool roof         48         74      0.776      0.701      0.754      0.474
          generic roof         48        291      0.704      0.849      0.844      0.573
Speed: 0.2ms preprocess, 4.2ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79b10e1c3f40>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [ ]:
model = YOLO('/content/runs/detect/train/weights/best.pt')
results = model.predict(source=data_path + 'test/*.png',
                        save=True,
                         conf=0.5,
                         iou=0.3,
                        line_width=2)


image 1/10 /content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/data/test/test_01.png: 640x384 1 cool roof, 13 generic roofs, 141.0ms
image 2/10 /content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/data/test/test_02.png: 640x352 5 generic roofs, 99.2ms
image 3/10 /content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/data/test/test_03.png: 640x608 1 cool roof, 33 generic roofs, 76.0ms
image 4/10 /content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/data/test/test_04.png: 512x640 2 cool roofs, 22 generic roofs, 72.9ms
image 5/10 /content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/data/test/test_05.png: 288x640 10 generic roofs, 74.6ms
image 6/10 /content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝ᄐ

In [ ]:
pred_path = glob.glob('/content/runs/detect/predict/*.png') # 학습 데이터 추가하기 전

for path in pred_path:
    plt.imshow(image.load_img(path))
    plt.show()

Output hidden; open in https://colab.research.google.com to view.

#### 2) 학습 데이터 추가 후

In [102]:
model = YOLO(model='yolov8n.pt', task='detect')

In [103]:
model.train(data=data_path + 'data.yaml',
            epochs=300,
            patience=20,
            pretrained=True,
            verbose=True,
            seed=2024
            )

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/data/data.yaml, epochs=300, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train14, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=2024, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frame

train: Scanning /content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/data/train/labels.cache... 760 images, 0 backgrounds, 0 corrupt: 100%|██████████| 760/760 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 1683, len(boxes) = 5535. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/data/valid/labels.cache... 191 images, 0 backgrounds, 0 corrupt: 100%|██████████| 191/191 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 528, len(boxes) = 1427. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


...

    140/300      3.73G     0.9192     0.5697      1.103         58        640: 100%|██████████| 48/48 [00:22<00:00,  2.15it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:02<00:00,  2.02it/s]

                   all        191       1427      0.911       0.85      0.909      0.646



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    141/300      3.64G     0.9293     0.5627      1.113        168        640:  15%|█▍        | 7/48 [00:03<00:21,  1.91it/s]


KeyboardInterrupt: 

In [92]:
model = YOLO('/content/runs/detect/train9/weights/best.pt')
results = model.predict(source='/content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/test/*.png',
                        save=True,
                        conf=0.5,
                        iou=0.3,
                        line_width=2)


image 1/10 /content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/test/test_01.png: 640x384 1 cool roof, 5 generic roofs, 36.8ms
image 2/10 /content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/test/test_02.png: 640x352 4 generic roofs, 25.8ms
image 3/10 /content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/test/test_03.png: 640x608 4 cool roofs, 14 generic roofs, 26.0ms
image 4/10 /content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/test/test_04.png: 512x640 15 generic roofs, 26.1ms
image 5/10 /content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/test/test_05.png: 288x640 3 generic roofs, 27.3ms
image 6/10 /content/drive/MyDrive/취업/KT_AIVLE_SCHOOL/week8/Mini_project4/2024.04.11_미니프로젝트 4차_실습자료/test/test_06.png: 480x64

In [93]:
pred_path = glob.glob('/content/runs/detect/predict7/*.png') # 학습 데이터 추가 후

for path in pred_path:
    plt.imshow(image.load_img(path))
    plt.show()

Output hidden; open in https://colab.research.google.com to view.